In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tilespec import TileSpec,Transform,AffineModel,ResolvedTileSpecCollection, ResolvedTileSpecMap
from renderapi import Render
import os
import json
import numpy as np

In [19]:
owner = 'Forrest'
project = 'M247514_Rorb_1'

json_dir = '/media/neurodata/6tbSFN/data/M247514_Rorb_1/processed/aligned_tilespecs/'
aligned_prefix = 'ALIGNED2_'
aligned_prefix_new = 'ALIGNED3_'
json_dir = '/media/neurodata/6tbSFN/data/M247514_Rorb_1/processed/aligned_tilespecs/'
client_scripts = '/home/neurodata/Projects/render/render-ws-java-client/src/main/scripts/'
host = '192.168.0.150'
port = 8080
render = Render(host,port,owner,project,default_client_scripts=client_scripts)


In [25]:
json_files = [os.path.join(json_dir,f) for f in os.listdir(json_dir) if aligned_prefix in f]
for f in json_files:
    d=json.load(open(f,'r'))
    tilespecs = [TileSpec(json=ts) for ts in d]
    for ts in tilespecs:
        ts.imageUrl=ts.imageUrl.replace('/nas/data/','/media/neurodata/6tbSFN/data/')
        ts.maskUrl = None
    fnew = f.replace(aligned_prefix,aligned_prefix_new)
    json.dump([ts.to_dict() for ts in tilespecs],open(fnew,'w'))
    #rts = ResolvedTileSpecMap(tilespecs=tilespecs,transforms=[])
    inputStack=f[f.find(aligned_prefix):-5]
    outputStack = inputStack.replace(aligned_prefix,aligned_prefix_new)
    r=render.create_stack(outputStack)
    render.set_stack_state(outputStack,'LOADING')
    #r2=render.put_resolved_tilespecs(outputStack,json.dumps(rts.to_dict()))
    
    render.import_jsonfiles(outputStack,[fnew],verbose=True,client_scripts=client_scripts)
    #break

stack version2 http://192.168.0.150:8080/render-ws/v1/owner/Forrest/project/M247514_Rorb_1/stack/ALIGNED3_DAPI_1 {'stackResolutionZ': 1, 'mipmapPathBuilder': {'numberOfLevels': 0}, 'createTimestamp': '2016-41-14T14:41:36.00Z', 'stackResolutionX': 1, 'materializedBoxRootPath': 'string', 'cycleStepNumber': 1, 'cycleNumber': 1, 'stackResolutionY': 1, 'versionNotes': ''}
http://192.168.0.150:8080/render-ws/v1/owner/Forrest/project/M247514_Rorb_1/stack/ALIGNED3_DAPI_1/state/LOADING
http://192.168.0.150:8080/render-ws/v1/owner/Forrest/project/M247514_Rorb_1/stack/ALIGNED3_DAPI_1/state/LOADING
['/home/neurodata/Projects/render/render-ws-java-client/src/main/scripts/import_json.sh', '--baseDataUrl', 'http://192.168.0.150:8080/render-ws/v1', '--owner', 'Forrest', '--project', 'M247514_Rorb_1', '--stack', 'ALIGNED3_DAPI_1', '/media/neurodata/6tbSFN/data/M247514_Rorb_1/processed/aligned_tilespecs/Sharmishtaas_M247514_Rorb_1_ALIGNED3_DAPI_1.json']

  Running: /home/neurodata/Projects/render/deploy

In [14]:
json_files[0]

'/media/neurodata/6tbSFN/data/M247514_Rorb_1/processed/aligned_tilespecs/Sharmishtaas_M247514_Rorb_1_ALIGNED2_DAPI_1.json'

In [92]:

js=rts.to_dict()


In [93]:
print json.dumps(js)[0:100]

{"transformIdToSpecMap": {}, "tileIdToSpecMap": {"101000004010000": {"minIntensity": 1575.0, "layout


In [74]:
print js.keys()
print [type(js[key]) for key in js.keys()]
print js['tileCount']
print type(js)
print len(js['tileSpecs'])


['tileCount', 'transformSpecs', 'tileSpecs', 'transformCount']
[<type 'int'>, <type 'list'>, <type 'list'>, <type 'int'>]
1836
<type 'dict'>
1836


In [64]:
inputStacks = [st for st in render.get_stacks_by_owner_project(inputOwner,inputProject) if st.startswith(inputPrefix)]

NameError: name 'inputOwner' is not defined

In [24]:
def define_aligned_tilespecs(prealignedStack,postalignedStack,inputStack):
    zvalues = render.get_z_values_for_stack(inputStack)
    #zvalues = [22.0]
    finaltilespecs = []
    for z in zvalues:
        prealignedTS = render.get_tile_specs_from_z(prealignedStack,z,owner=prealignedOwner)[7]
        postalignedTS = render.get_tile_specs_from_z(postalignedStack,z,owner=postalignedOwner)
        postalignedTS =[ts for ts in postalignedTS if ts.tileId==prealignedTS.tileId][0]
        tform_W_to_R = prealignedTS.tforms

        tform_R_to_W = list(tform_W_to_R)
        tform_R_to_W.reverse()
        tform_R_to_W = [tf.invert() for tf in tform_R_to_W]

        tform_W_to_A = postalignedTS.tforms

        tform_R_to_A = tform_R_to_W + tform_W_to_A
        #print tform_R_to_A

        inputTS = render.get_tile_specs_from_z(inputStack,z,owner=inputOwner)
        for ts in inputTS:
            ts.tforms+=tform_R_to_A
            finaltilespecs.append(ts)
    return finaltilespecs

In [25]:
for inputStack in inputStacks:
    inputchannel = inputStack.replace(inputPrefix,'')
    outputStack = outputPrefix + inputchannel
    print inputStack,outputStack
    
    finaltilespecs = define_aligned_tilespecs(prealignedStack,postalignedStack,inputStack)
    json_tilespecs=[ts.to_dict() for ts in finaltilespecs]
    if not os.path.isdir(outputDir):
        os.makedirs(outputDir)
    outjson = os.path.join(outputDir,'%s_%s_%s.json'%(inputOwner,inputProject,outputStack))
    json.dump(json_tilespecs,open(outjson,'w'),indent=4)
    print outputStack,outjson
    render.delete_stack(outputStack,owner=outputOwner,project=outputProject)
    render.create_stack(outputStack,owner=outputOwner,project=outputProject)
    
    render.import_jsonfiles(outputStack,[outjson],owner=outputOwner,project=outputProject)

REGISTERED_FLATFIELD_FIX_MBP ALIGNED2_MBP
http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/REGISTERED_FLATFIELD_FIX_MBP/zValues/
ALIGNED2_MBP /nas/data/M247514_Rorb_1/processed/aligned_tilespecs/Sharmishtaas_M247514_Rorb_1_ALIGNED2_MBP.json

http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/ALIGNED2_MBP/state/LOADING
http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/ALIGNED2_MBP/state/COMPLETE
REGISTERED_FLATFIELD_FIX_DAPI_3 ALIGNED2_DAPI_3
http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/project/M247514_Rorb_1/stack/REGISTERED_FLATFIELD_FIX_DAPI_3/zValues/
ALIGNED2_DAPI_3 /nas/data/M247514_Rorb_1/processed/aligned_tilespecs/Sharmishtaas_M247514_Rorb_1_ALIGNED2_DAPI_3.json

http://ibs-forrestc-ux1.corp.alleninstitute.org:8081/render-ws/v1/owner/Sharmishtaas/pro

In [58]:
import time

In [59]:
t=time.time()

In [73]:
time.strftime('%Y-%M-%dT%H:%M:%S.00Z')

'2016-46-14T09:46:21.00Z'

In [65]:
time.strftime?